In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('a1_RestaurantReviews_HistoricDump.tsv', delimiter='\t')

In [8]:
import re
import nltk
import pickle
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/subhashbishnoi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# Recreate the TextPreprocessor class
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ps = PorterStemmer()
        self.stopwords = set(stopwords.words('english'))
        self.stopwords.remove('not')

    def fit(self, X, y=None):
        return self  # No fitting required

    def transform(self, X):
        corpus = []
        for review in X:
            review = re.sub(r'[^a-zA-Z]', ' ', review)
            review = review.lower()
            review = review.split()
            review = [self.ps.stem(word) for word in review if word not in self.stopwords]
            corpus.append(' '.join(review))
        return corpus


In [24]:
pipeline = Pipeline([
    ('text_preprocessor', TextPreprocessor())
])


In [25]:
X = df['Review']
y = df.iloc[:, -1].values  # Assuming last column is the label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('text_preprocessor', TextPreprocessor()),  
    ('vectorizer', CountVectorizer(max_features=1420)),  
    ('classifier', LogisticRegression())  
])

In [26]:
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

In [27]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.7667


In [28]:
model_path = 'sentiment_analysis_pipeline.pkl'
pickle.dump(pipeline, open(model_path, "wb"))

print(f'Model saved as {model_path}')

Model saved as sentiment_analysis_pipeline.pkl


In [29]:
loaded_pipeline = pickle.load(open('sentiment_analysis_pipeline.pkl', 'rb'))
prediction = loaded_pipeline.predict(["ek dum solid."])
print(prediction)

[1]
